# Feature Engineering test ground
![Reverse Engineer](https://crystal-cdn3.crystalcommerce.com/photos/6337507/en_UJy7SBBy4F.png)

In [54]:
import numpy as np
import pandas as pd
import math
from master_transmuter import *
from unit_tests import *

In [4]:
cd ..

C:\Users\alexg\Galvanize\mystic-speculation


In [31]:
X = pd.read_csv('data/all_vintage_cards-rare_recent.csv', low_memory=False)

In [40]:
cards = [x.split('//') for x in X['type_line'].unique()]
cards[:10]

[['Legendary Creature — Minotaur Cleric'],
 ['Sorcery'],
 ['Legendary Creature — Human Cleric'],
 ['Land'],
 ['Legendary Artifact Creature — Construct'],
 ['Legendary Artifact'],
 ['Artifact Creature — Assembly-Worker'],
 ['Legendary Artifact — Equipment'],
 ['Artifact'],
 ['Artifact — Equipment']]

In [48]:
card_types = set(['Creature','Land','Instant','Sorcery','Enchantment','Artifact','Planeswalker'])
sub_types = set()
type_mods = set()

for card in cards:
    for subcard in card:
        types = subcard.split(' — ')
        type_mods.update(set(types[0].split()) - card_types)
        try:
            sub_types.update(set(types[1].split()))
        except:
            pass
print(typetype_mods)
print(sub_types)

{'Snow', 'Tribal', 'World', 'Legendary'}
{'Merfolk', 'Saga', 'Kraken', 'Dwarf', 'Thrull', 'Snake', 'Advisor', 'Lizard', 'Spellshaper', 'Orgg', 'Goblin', 'Nightstalker', 'Fox', 'Rat', 'Cephalid', 'Gorgon', 'Forest', 'Squirrel', 'Rogue', 'Ox', 'Archon', 'Fish', 'Griffin', 'Elemental', 'Dragon', 'Berserker', 'Wolf', 'Soltari', 'Scout', 'Elephant', 'Rabbit', 'Gnome', 'Hound', 'Pirate', 'Specter', 'Djinn', 'Badger', 'Drone', 'Barbarian', 'Faerie', 'Hellion', 'Eldrazi', 'Bringer', 'Ajani', 'Wall', 'Kavu', 'Shapeshifter', 'Giant', 'Pilot', 'Yeti', 'Cleric', 'Thopter', 'Mercenary', 'Spike', 'Boar', 'Myr', 'Plains', 'Curse', 'Beast', 'Angel', 'Artificer', 'Swamp', 'Shade', 'Whale', 'Orc', 'Trap', 'Crab', 'Druid', 'Kor', 'Imp', 'Crocodile', 'Gargoyle', 'Serpent', 'Horse', 'Volver', 'Nightmare', 'Cyclops', 'Metathran', 'Worm', 'Zombie', 'Fortification', 'Incarnation', 'Moonfolk', 'Treefolk', 'Illusion', 'Warrior', 'Spirit', 'Dreadnought', 'Nomad', 'Assassin', 'Rhino', 'Masticore', 'Construct', 'M

In [51]:
cards[0]

['Legendary Creature — Minotaur Cleric']

In [52]:
subcard = cards[1]
types = subcard[0].split(' — ')
types

['Sorcery']

In [56]:
typeline = TypelineTransformer()
typeline_test = typeline.transform(X)
typeline_test.head()

[autoreload of master_transmuter failed: Traceback (most recent call last):
  File "C:\Users\alexg\Anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\alexg\Anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 368, in superreload
    module = reload(module)
  File "C:\Users\alexg\Anaconda3\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\Users\alexg\Anaconda3\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 600, in _exec
AttributeError: 'NoneType' object has no attribute 'name'
]


KeyError: "['card_types' 'sub_types'] not in index"

In [2]:
cd ..

C:\Users\alexg\Galvanize\mystic-speculation


In [ ]:
cd scrape

In [ ]:
scraping_progress(['mythic', 'rare'])

In [50]:
cards_df = pd.read_csv('scrape/all_vintage_cards.csv')
creature = cards_df[cards_df['name']=='Arcanis the Omnipotent'].iloc[0]
spell = cards_df[cards_df['name']=='Momentary Blink'].iloc[0]
weird = cards_df[cards_df['name']=='Tarmogoyf'].iloc[0]
tests = pd.concat([creature, spell, weird], axis=1).T
tests

FileNotFoundError: File b'scrape/all_vintage_cards.csv' does not exist

In [ ]:
pt_mask = np.array([False, False, True])
tests['variable_pt'] = True
tests['variable_pt'][~pt_mask] = False
tests

In [ ]:
pt_mask = np.array([True, False, False])
tests['p:t'] = tests[pt_mask]['power'].astype('int')/tests[pt_mask]['toughness'].astype('int')
tests

In [ ]:
def pt_type(row):
    if (type(row['power']) == type('str')) and (type(row['toughness']) == type('str')):
        if '*' in row['power']+row['toughness']:
            return 'variable'
        return 'static'
    return 'none'
def power_to_int(row):
    if row['pt_type']=='static': 
        return int(row['power'])
    else:
        return row['power']
def tough_to_int(row):
    if row['pt_type']=='static':
        return int(row['toughness'])
    else:
        return row['toughness']
df = tests.copy()
# Create pt_type feature, convert static pts to ints
df['pt_type'] = df.apply(pt_type, axis=1)
df['power'] = df.apply(power_to_int, axis=1)
df['toughness'] = df.apply(tough_to_int, axis=1)

mask = df['pt_type']=='static'

df['p:t'] = df[mask]['power']/df[mask]['toughness']
df['p+t'] = df[mask]['power']+df[mask]['toughness']
df['p*t'] = df[mask]['power']*df[mask]['toughness']
df['sqrt_pt'] = math.sqrt(df[mask]['p*t'])
df['avg_pt'] = (df[mask]['power']+df[mask]['toughness'])/2
df['cmc:p+t'] = df[mask]['cmc']/df[mask]['p+t']
df['cmc:p*t'] = df[mask]['cmc']/df[mask]['sqrt_pt']
df

In [ ]:
df['p*t'].apply(np.sqrt)

In [ ]:
math.sqrt((df[mask]['power']*df[mask]['toughness']))

In [ ]:
from unit_tests import *

In [ ]:
test_feature_engineering(cards_df)

In [49]:
from scrape.query import *

[autoreload of master_transmuter failed: Traceback (most recent call last):
  File "C:\Users\alexg\Anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\alexg\Anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 368, in superreload
    module = reload(module)
  File "C:\Users\alexg\Anaconda3\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\Users\alexg\Anaconda3\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 600, in _exec
AttributeError: 'NoneType' object has no attribute 'name'
]


In [ ]:
test_cards = cards_df[cards_df['rarity']=='mythic']
test_cards[['recent_date','recent_price']] = test_cards.apply(get_recent_price, axis=1).apply(pd.Series)

In [130]:
cards_df[['recent_date','recent_price']] = cards_df.apply(get_recent_price, axis=1).apply(pd.Series)

ProgrammingError: ('(psycopg2.ProgrammingError) relation "uncommon_price_history_2" does not exist\nLINE 1: ...cardname, ph.setname, ph.timestamp, ph.price from uncommon_p...\n                                                             ^\n', 'occurred at index 53') [SQL: "select ph.cardname, ph.setname, ph.timestamp, ph.price from uncommon_price_history_2 ph,      (select max(timestamp) as lastdate       from uncommon_price_history_2 ph2       where ph2.cardname='Kitchen Finks' and ph2.setname='Ultimate Masters') mostrecent where ph.timestamp = mostrecent.lastdate "] (Background on this error at: http://sqlalche.me/e/f405)

In [116]:
test_card.apply(get_recent_price, axis=1)

('Polyraptor', 'Rivals of Ixalan', '1542070311053', 2.3)


2857    (1542070311053, 2.3)
dtype: object

In [122]:
test_card.apply(get_recent_price, axis=1)[:]

('Polyraptor', 'Rivals of Ixalan', '1542070311053', 2.3)


2857    (1542070311053, 2.3)
dtype: object

In [ ]:
write_recent_prices(cards_df, ['mythic', 'rare'])

In [1]:
ls

 Volume in drive C is OS
 Volume Serial Number is 606C-8C38

 Directory of C:\Users\alexg\Galvanize\mystic-speculation\model

11/15/2018  09:17 PM    <DIR>          .
11/15/2018  09:17 PM    <DIR>          ..
11/14/2018  05:34 PM    <DIR>          .ipynb_checkpoints
11/15/2018  06:29 PM    <DIR>          __pycache__
11/15/2018  06:22 PM             6,066 master_transmuter.py
11/15/2018  09:17 PM            31,720 Reverse_Engineer.ipynb
               2 File(s)         37,786 bytes
               4 Dir(s)  10,818,215,936 bytes free
